In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import tensorflow_hub as hub

tf.logging.set_verbosity(tf.logging.ERROR)
!pip install bert-tensorflow --quiet

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
os.chdir("./gdrive/My Drive/NLU/Projects/project_2/")
print(os.getcwd())
print(os.listdir())


from bert_sct_utils import create_tokenizer_from_hub_module, SctProcessor
from bert_sct_utils import model_fn_builder, file_based_input_fn_builder, file_based_convert_examples_to_features

W0603 10:53:26.181705 140257537501056 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Mounted at /content/gdrive/
/content/gdrive/My Drive/NLU/Projects/project_2
['.ipynb_checkpoints', 'report', 'venv', 'data', 'setup.py', 'preprocess_data.py', 'docs', 'data_pp', '__pycache__', 'results', '.idea', 'bert', 'bert_cloze.py', 'bert_utils.py', 'bert_cloze_utils.py', 'bert_sct_utils.py', 'bert_sct.py', 'bert_sct.ipynb']


In [0]:
flags = tf.flags
FLAGS = flags.FLAGS

flags.DEFINE_string("data_dir", "./data_pp/", "The input data dir. Should contain the .tsv files (or other data files).")
flags.DEFINE_string("output_dir", "./results/", "The output directory where the model checkpoints will be written.")

# BERT
flags.DEFINE_string("bert_model_hub", "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", "Pretrained BERT.")

# Training or evaluation
flags.DEFINE_bool("do_train", True, "Whether to run training.")
flags.DEFINE_bool("do_eval", True, "Whether to run eval on the dev set.")
flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

flags.DEFINE_integer("max_seq_length", 128, "Max. length after tokenization. If shorter padded, else truncated.")
flags.DEFINE_integer("batch_size", 16, "Total batch size for training.")
flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")
flags.DEFINE_float("num_train_epochs", 0.3, "Total number of training epochs to perform.")
flags.DEFINE_float("warmup_proportion", 0.01, "Proportion of training to perform linear learning rate warmup for.")
flags.DEFINE_integer("save_checkpoints_steps", 1000, "How often to save the model checkpoint.")
flags.DEFINE_integer("iterations_per_loop", 1, "How many steps to make in each estimator call.")

In [0]:
def main(_):
    tf.logging.set_verbosity(tf.logging.INFO)

    tf.gfile.MakeDirs(FLAGS.output_dir)

    processor = SctProcessor()
    label_list = processor.get_labels()
    tokenizer = create_tokenizer_from_hub_module(FLAGS.bert_model_hub)

    train_examples = None
    num_train_steps = None
    num_warmup_steps = None
    if FLAGS.do_train:
        train_examples = processor.get_train_examples(FLAGS.data_dir)
        num_train_steps = int(len(train_examples) / FLAGS.batch_size * FLAGS.num_train_epochs)
        num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

    run_config = tf.estimator.RunConfig(model_dir=FLAGS.output_dir,
                                        save_summary_steps=10,
                                        save_checkpoints_steps=1000)

    model_fn = model_fn_builder(bert_model_hub=FLAGS.bert_model_hub,
                                num_labels=len(label_list),
                                learning_rate=FLAGS.learning_rate,
                                num_train_steps=num_train_steps,
                                num_warmup_steps=num_warmup_steps)

    params = {
        "batch_size": FLAGS.batch_size,
    }

    estimator = tf.estimator.Estimator(model_fn=model_fn,
                                       config=run_config,
                                       params=params)
    # ---------------------------------------------------------------------------------------------------------------- #
    # Traning
    if FLAGS.do_train:
        train_file = os.path.join(FLAGS.data_dir, "train.tf_record")
        file_based_convert_examples_to_features(train_examples,
                                                label_list,
                                                FLAGS.max_seq_length,
                                                tokenizer,
                                                train_file)

        tf.logging.info("***** Running training *****")
        tf.logging.info("  Num examples = %d", len(train_examples))
        tf.logging.info("  Batch size = %d", FLAGS.batch_size)
        tf.logging.info("  Num steps = %d", num_train_steps)

        train_input_fn = file_based_input_fn_builder(input_file=train_file,
                                                     seq_length=FLAGS.max_seq_length,
                                                     is_training=True,
                                                     drop_remainder=False)

        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    # ---------------------------------------------------------------------------------------------------------------- #
    # Validation
    if FLAGS.do_eval:
        eval_examples = processor.get_dev_examples(FLAGS.data_dir)

        eval_file = os.path.join(FLAGS.data_dir, "eval.tf_record")
        file_based_convert_examples_to_features(eval_examples,
                                                label_list,
                                                FLAGS.max_seq_length,
                                                tokenizer,
                                                eval_file)

        tf.logging.info("***** Running evaluation *****")
        tf.logging.info("  Num examples = %d", len(eval_examples))

        eval_steps = int(len(eval_examples) // FLAGS.batch_size)

        eval_input_fn = file_based_input_fn_builder(input_file=eval_file,
                                                    seq_length=FLAGS.max_seq_length,
                                                    is_training=False,
                                                    drop_remainder=False)

        result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

        output_eval_file = os.path.join(FLAGS.output_dir, "eval_results.txt")
        with tf.gfile.GFile(output_eval_file, "w") as writer:
            tf.logging.info("***** Eval results *****")
            for key in sorted(result.keys()):
                tf.logging.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

    # ---------------------------------------------------------------------------------------------------------------- #
    # Test
    if FLAGS.do_predict:
        predict_examples = processor.get_test_examples(FLAGS.data_dir)

        predict_file = os.path.join(FLAGS.data_dir, "predict.tf_record")
        file_based_convert_examples_to_features(predict_examples,
                                                None,
                                                FLAGS.max_seq_length,
                                                tokenizer,
                                                predict_file)

        tf.logging.info("***** Running prediction*****")
        tf.logging.info("  Num examples = %d", len(predict_examples))

        predict_input_fn = file_based_input_fn_builder(input_file=predict_file,
                                                       seq_length=FLAGS.max_seq_length,
                                                       is_training=False,
                                                       drop_remainder=False)

        result = estimator.predict(input_fn=predict_input_fn)

        output_predict_file = os.path.join(FLAGS.output_dir, "test_results.tsv")
        with tf.gfile.GFile(output_predict_file, "w") as writer:
            num_written_lines = 0
            tf.logging.info("***** Predict results *****")
            for (i, prediction) in enumerate(result):
                probabilities = prediction["probabilities"]

                output_line = "\t".join(str(class_probability) for class_probability in probabilities) + "\n"
                writer.write(output_line)
                num_written_lines += 1


if __name__ == "__main__":
    tf.app.run()

Instructions for updating:
Colocations handled automatically by placer.


W0603 10:53:52.756721 140257537501056 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 10:53:55.121967 140257537501056 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Using config: {'_model_dir': './results/', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f901072d048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

I0603 10:54:31.101579 140257537501056 session_manager.py:493] Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 12 into ./results/model.ckpt.


I0603 10:54:39.975696 140257537501056 basic_session_run_hooks.py:594] Saving checkpoints for 12 into ./results/model.ckpt.


INFO:tensorflow:loss = 0.3336231, step = 13


I0603 10:55:49.704315 140257537501056 basic_session_run_hooks.py:249] loss = 0.3336231, step = 13
